In [ ]:
%%time
!pip install -r requrements.txt
# coding: utf-8
import nltk
import time
import numpy as np
from scipy.sparse import csr_matrix, save_npz, load_npz, lil_matrix
from sklearn.preprocessing import normalize
from nltk.stem.snowball import SnowballStemmer
from string import punctuation
from collections import OrderedDict
#np.set_printoptions(threshold=np.inf)
import os

In [37]:
words_check = sorted([
    'любой', 'человек'
])

soften = 0.05

In [ ]:
mpz_file = 'book_mm.npz'
M_learned = load_npz(mpz_file)
lemm_learned = open('lemlist.txt', 'r').read().splitlines()

In [ ]:
stemmer = SnowballStemmer("russian")

lemm_check = sorted([stemmer.stem(word) for word in words_check])

word_match = {}
for i in words_check:
    word_match[stemmer.stem(i)] = i

new_lem = list(sorted(set(lemm_check) & set(lemm_learned)))

print(word_match)
print(lemm_check)
print(new_lem)

In [ ]:
order = len(lemm_check)
M = np.ndarray((order, order))
for i in lemm_check:
    #print(i)
    x_new = lemm_check.index(i)
    if i in new_lem:
        x_learn = lemm_learned.index(i)
        for j in lemm_check:
            if j in new_lem:
                y_new = lemm_check.index(j)
                y_learn = lemm_learned.index(j)
                #print("new coord: {} {} , old coord: {} {}".format(x_new, y_new, x_learn, y_learn))
                M[x_new, y_new] = M_learned[x_learn, y_learn]
            else:
                pass
    else:
        #print(i)
        M[x_new,:] = [1/order for x in range(order)]
        

In [ ]:
for i in set(np.where(M == 0)[0]):
    M[i,:] += soften

#with np.nditer(M, op_flags=['readwrite']) as items:
#    for i in items:
#        if i == 0:
#            i[...] += 1.0e-10
M_normalized = normalize(M, norm='l1', axis=1, copy=False)

In [ ]:
#for a in lemm_check:
#    arr = [x for x in M[lemm_check.index(a)].flat]
#    print(a,len(arr), type(arr) ,arr)
#print('')    
#for b in lemm_check:
#    arr = [x for x in M_normalized[lemm_check.index(b)].flat]
#    print(b,len(arr), type(arr) ,arr)

In [ ]:
def sentence(mat, depth, lemms, curword=None, final=[], words=None):
    if depth != 0:
        depth -= 1
        if not curword:
            nextword = np.random.choice(lemms)
        else:
            arr = [x for x in  mat[lemms.index(curword)].flat]
            #nextword=curword
            nextword=np.random.choice(lemms,p=arr)
            final.append(words[nextword])

        sentence(mat, depth, lemms, nextword, final, words)
    return(' '.join(final))
    


In [ ]:
for x in range(50):
    print(sentence(M_normalized, 16, lemm_check, final=[], words=word_match))
